# AB test 

A/B testing is the research method that allows you to find out the effect of a particular change in the product. The study shows which of the two versions of the product or offer gives greater effect on the selected metrics and if it is statistically significant.  

<ul>
  <li><a href="#creation-of-a-new-test-dataset-with-synthetic-data">Creation of a new test dataset with synthetic data.
  <li><a href="#ab-test">AB test.
  <li><a href="#cuped-classic-covariate-adjustment-for-variance-reduction">CUPED: Classic Covariate Adjustment for Variance Reduction.
  <li><a href="#cupac-advanced-covariate-adjustment">CUPAC: Advanced Covariate Adjustment.
  <li><a href="#additional-tests-in-ab-test">Additional tests in AB Test.
  <li><a href="#abn-test">ABn Test.
</ul>

In [1]:
import random

from hypex import ABTest
from hypex.dataset import Dataset, InfoRole, TargetRole, TreatmentRole

## Creation of a new test dataset with synthetic data. 

In order to be able to work with our data in HypEx, first we need to convert it into `dataset`. It is important to mark the data fields by assigning the appropriate `roles`:
- FeatureRole: a role for columns that contain features or predictor variables. Our split will be based on them. Applied by default if the role is not specified for the column.
- TreatmentRole: a role for columns that show the treatment or intervention.
- TargetRole: a role for columns that show the target or outcome variable.
- InfoRole: a role for columns that contain information about the data, such as user IDs. 

In [2]:
from hypex.utils.tutorial_data_creation import DataGenerator
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
gen1 = DataGenerator(
    n_samples=2000,
    distributions={
        "X1": {"type": "normal", "mean": 0, "std": 1},
        "X2": {"type": "bernoulli", "p": 0.5},
        "y0": {"type": "normal", "mean": 5, "std": 1},
    },
    time_correlations={"X1": 0.2, "X2": 0.1, "y0": 0.6},
    effect_size=2.0,
    seed=7
)
df = gen1.generate()
df = df.drop(columns=['y0', 'z', 'U', 'D', 'y1', 'y0_lag_2'])

data = Dataset(
    roles={
        "d": TreatmentRole(),
        "y": TargetRole(),
    },
    data=df,
    default_role=InfoRole()
 )

The roles' data types can be assigned automatically as shown below. Also, the fields, which were not marked, receive Feature role by default.

In [4]:
data.roles

{'d': Treatment(<class 'int'>),
 'y': Target(<class 'float'>),
 'X1': Info(<class 'float'>),
 'X1_lag': Info(<class 'float'>),
 'X2': Info(<class 'int'>),
 'X2_lag': Info(<class 'int'>),
 'y0_lag_1': Info(<class 'float'>)}

## AB test
Then we select one of the pre-assembled pipelines, in our case `ABTest`. Also, a custom pipeline can be created based on your specific needs and requirements with custom executors.
After that we wrap our prepared `dataset` into `ExperimentData` to be able to run experiments on it and then execute the test with this data passed as the argument.

In [5]:
test = ABTest()
result = test.execute(data)

### Experiment results
To show the report with summary of the test we run the `resume` method of the output of the experiment.

It displays the results of the test in the form of a table with the following columns:
- `feature`: name of the target feature, change of which we want to analyze.
- `group`: name of the test group we compare with the control group.
- `TTest pass`: result of the TTest, if it is significant or not.
- `TTest p-value`: p-value of the TTest shows the probability of obtaining the result when the null hypothesis is true. The lower the value the more significant the result is.
- `control mean`: the mean of the feature value across the control group.
- `test mean`: the mean of the feature value across the test group.
- `difference`: the difference between the mean of the test group and the mean of the control group.
- `difference %`: the normalized difference between the mean of the test group and the mean of the control group.

In [6]:
result.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.815482,7.827936,3.012454,62.557684,OK,1.895971e-157


In [7]:
result.sizes

,control size,test size,control size %,test size %,group
1,1352,648,67.6,32.4,1


In [8]:
result.multitest

"There was less than three groups or multitest method wasn't provided"

## CUPED: Classic Covariate Adjustment for Variance Reduction

CUPED (Controlled Experiments Using Pre-Experiment Data) is a classic method for variance reduction in A/B tests. It uses historical or auxiliary features (such as target lags) to adjust the target variable and increase the statistical power of the test.

In HypEx, to apply CUPED, simply specify the corresponding features via the `cuped_features` parameter in `ABTest` or directly in the CUPEDTransformer. As a result, a new column (e.g., `y_cuped`) is created, which is automatically used for analysis.

Example of running ABTest with CUPED adjustment:

In [9]:
test = ABTest(cuped_features={'y': 'y0_lag_1'})
result = test.execute(data)

In [10]:
result.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.815482,7.827936,3.012454,62.557684,OK,1.895971e-157
1,y_cuped,1,4.924818,7.599815,2.674998,54.316686,OK,6.901190e-188


### Variance Reduction Report for CUPED

After applying CUPED, you can now view the variance reduction achieved. This shows the percentage reduction in variance for the adjusted target variable, which indicates the effectiveness of the CUPED adjustment. Access it via the `variance_reduction_report` property:

In [11]:
result.variance_reduction_report

,Transformed Metric Name,Variance Reduction (%)
0,y_cuped,31.869217


## CUPAC: Advanced Covariate Adjustment

CUPAC (Covariate-Updated Pre-Analysis Correction) is an advanced method for variance reduction in A/B testing. It extends the CUPED approach by allowing flexible model selection (linear, ridge, lasso, or CatBoost regression) to adjust the target variable using historical or auxiliary features. This can lead to more accurate and powerful statistical tests.

To use CUPAC in HypEx, specify the `cupac_features` argument in `ABTest`, including the target and covariate columns, and optionally the `cupac_model` argument. You can pass a string (e.g. 'linear') or a list of model names (e.g. ['linear', 'ridge', 'lasso']) to `cupac_model`. The result is a new target column (e.g., `y_cupac`) automatically added to your dataset and used in the analysis.

Below is an example of running ABTest with CUPAC adjustment:

In [12]:
# Run ABTest with CUPAC adjustment
# You can pass a string (e.g. 'linear') or a list of model names (e.g. ['linear', 'ridge', 'lasso']) to cupac_model.
test = ABTest(cupac_features={
    'y': ['y0_lag_1', 'X1_lag', 'X2_lag'],
}, cupac_model=['linear', 'ridge', 'lasso'])
result = test.execute(data)

In [13]:
result.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.815482,7.827936,3.012454,62.557684,OK,1.895971e-157
1,y_cupac,1,5.050837,7.336885,2.286048,45.260764,OK,4.379773e-160


### Variance Reduction Report for CUPAC

Similar to CUPED, CUPAC also provides a variance reduction report showing the percentage reduction in variance achieved by the model-based adjustment. This helps evaluate the effectiveness of the CUPAC transformation. Access it via the `variance_reduction_report` property:

In [14]:
result.variance_reduction_report

,Transformed Metric Name,Variance Reduction (%)
0,y_cupac,43.211131


## Additional tests in AB Test 

It is possible to add u-test and chi2-test in pipeline.

In [15]:
test = ABTest(additional_tests=['t-test', 'u-test', 'chi2-test'])
result = test.execute(data)

The additional columns are:
- `UTest pass`: result of the UTest, if it is significant or not.
- `UTest p-value`: p-value of the UTest shows the probability of obtaining the result when the null hypothesis is true. The lower the value the more significant the result is.
- `Chi2Test pass`: result of the Chi2Test, if it is significant or not.
- `Chi2Test p-value`: p-value of the Chi2Test shows the probability of obtaining the result when the null hypothesis is true. The lower the value the more significant the result is.

In [16]:
result.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value,UTest pass,UTest p-value
0,y,1,4.815482,7.827936,3.012454,62.557684,OK,1.895971e-157,OK,1.114725e-110
1,y_cupac,1,5.050837,7.336885,2.286048,45.260764,OK,4.379773e-160,OK,3.285338e-109


In [17]:
result.multitest

"There was less than three groups or multitest method wasn't provided"

In [18]:
result.sizes

,control size,test size,control size %,test size %,group
1,1352,648,67.6,32.4,1


## ABn Test 

Finally, we may run multiple ab tests with different methods.

In [19]:
test = ABTest(multitest_method="bonferroni")
result = test.execute(data)

In [20]:
result.resume

,feature,group,control mean,test mean,difference,difference %,TTest pass,TTest p-value
0,y,1,4.815482,7.827936,3.012454,62.557684,OK,1.895971e-157
1,y_cupac,1,5.050837,7.336885,2.286048,45.260764,OK,4.379773e-160


In [21]:
result.sizes

,control size,test size,control size %,test size %,group
1,1352,648,67.6,32.4,1


In [22]:
result.multitest

"There was less than three groups or multitest method wasn't provided"